In [1]:
import featuretools as ft
from dask import bag
from dask.diagnostics import ProgressBar
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
import os
from pathlib import Path
import re
import multiprocessing
import hvplot.pandas

import pickle
cpus = multiprocessing.cpu_count()

np.random.seed(42)
ft.__version__

/Users/bryan/.pyenv/versions/3.7.5/envs/capstone/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/bryan/.pyenv/versions/3.7.5/envs/capstone/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


'0.12.0'

## data path  
    Since we didn't have a unified directory structure for this two repositories, I hardcoded path here.
    Remember to change the path before running this notebook.

In [2]:
DATA_PATH = Path.cwd().parents[1] / "ArrestCaseDataPipeline/DCSC-Scraper"

In [3]:
Path.cwd().parents[0] / "ArrestCaseDataPipeline/DCSC-Scraper"

PosixPath('/Users/bryan/src/Sync/repos/github.com/donaldbraman/FTA-Modeling/ArrestCaseDataPipeline/DCSC-Scraper')

## number of parties we want to use

Subsampling before splitting data into two parts

In [4]:
num_party = 10000
party_files = DATA_PATH.glob("CMD_party*.csv")
# Now we have duplicates in party dataframe
party_df = pd.concat((pd.read_csv(f) for f in party_files), ignore_index=True).drop(columns="Party_Index").drop_duplicates().reset_index(drop=True)
party_df.head()

party_label            party_type party_akas  \
0                      BATES, TONY  Defendant (Criminal)        NaN   
1                     DAVID, SMITH  Defendant (Criminal)        NaN   
2                  KENYATTE, KAMAU  Defendant (Criminal)        NaN   
3                  SIMMS, PAMELA D  Defendant (Criminal)        NaN   
4  HOLLINGSWORTH, JENNIFER KRISTIN  Defendant (Criminal)        NaN   

      party_attorney          Case_ID  
0  WILLIAMS, JAMES E  2011 CMD 000001  
1     SIGNET, PAUL A  2011 CMD 000002  
2  WILLIAMS, JAMES E  2011 CMD 000005  
3     BARON, GREGG D  2011 CMD 000008  
4   STEWART, ANTHONY  2011 CMD 000012

In [5]:
# Sample num_party parties
subparty = party_df["party_label"].drop_duplicates().sample(n=num_party, random_state=42)

In [6]:
party_df = party_df[party_df["party_label"].isin(subparty)]
subcase = party_df["Case_ID"].unique()
print(f"There are {len(subcase)} cases for {num_party} consumers in our training set")

There are 17159 cases for 10000 consumers in our training set


Visualization #1

In [7]:
number_of_cases_per_party = party_df[party_df["Case_ID"].isin(subcase)].pivot_table(index="party_label",values="Case_ID",aggfunc="count")["Case_ID"].value_counts().rename("count")

In [8]:
number_of_cases_per_party.to_frame().reset_index().hvplot.bar(x="index", y="count", ylim=(0, number_of_cases_per_party.max()+100))

:Bars   [index]   (count)

## read summaries

In [9]:
# read in the summary information and create the summary entity
# Again, note that we must use the TimeIndex type for the file date. 
# This will allow us to only use prior cases to predict this case. 

summary_files = DATA_PATH.glob("CMD_summary*.csv")
summaries = pd.concat((pd.read_csv(f) for f in summary_files), ignore_index=True)
summaries = summaries[summaries["Case_ID"].isin(subcase)]

Split data into two parts

In [10]:
summaries = summaries.drop(columns=['Status_Date', 'Next_Event', 'Case_Status']).copy()
summaries['File_Date'] = pd.to_datetime(summaries['File_Date']).copy()
summaries['Hold_Out'] = summaries['File_Date'] > pd.to_datetime('12-31-2018')

In [11]:
print(f"We have {summaries['Hold_Out'].sum()} cases in hold out set, {len(summaries) - summaries['Hold_Out'].sum()} cases in current training set.")

We have 1099 cases in hold out set, 16060 cases in current training set.


In [12]:
# we'll need something like the following to filter all the dataframes: 

# The following function is to remove felony cases with detention events.
# Since we only consider CMD cases here, I didn't call it.
def remove_cases_with_detentions(df):
    df_copy = df.copy()
    df_copy["temp"] = df_copy["Result"].apply(lambda k: 1 if k in {'Preventive Detention', 'Defendant detained pending sentencing','Defendant Held in Other Jurisdiction'} else 0)
    pivot_df = df_copy.pivot_table(index="Case_ID", values="temp", aggfunc=np.sum)
    cases = set()
    for row in pivot_df.itertuples():
        if getattr(row, "temp"):
            cases.add(row.Index)
    return df[df["Case_ID"].isin(cases)]

Reduce the size of training set by setting the number of consumers we are going to train on.

## read docket

In [13]:
docket_files = DATA_PATH.glob("CMD_docket*.csv")
docket = pd.concat((pd.read_csv(f) for f in docket_files), ignore_index=True)
docket = docket[docket['Case_ID'].isin(subcase)]
docket = docket.drop(columns=['Image_Avail','Docket_Index'])
docket["Docket_Text"] = docket["Docket_Text"].fillna('')

In [14]:
# We start by reading in the docket bc we want to pull labels from it . 

docket['Date'] = pd.to_datetime(docket['Date'], errors='coerce').copy()
docket['fta'] = docket['Docket_Text'].str.contains('Defendant Failed to Appear') 
docket['fta'] = docket['fta'] * 1
docket["fta"] = docket["fta"].astype("int")

### extract charge type

In [15]:
def extract_charge_type(k):
    try:
        return re.findall(r"Charge Filed Charge #?1: (.+)", k)[0]
    except IndexError as e:
        return np.nan
docket["charge_type"] = docket["Docket_Text"].apply(extract_charge_type)
docket = docket.reset_index(drop=True)

### extract judge

In [16]:
def extract_judge(k):
    """ TODO: this way will extract judge like COURTROOM C-10, which is apparently wrong.
    """
    try:
        return re.findall(r"^Event Scheduled Event.+Judge\:\s(.+)\sLocation", k)[0]
    except IndexError as e:
        return np.nan

docket["judge"] = docket["Docket_Text"].apply(extract_judge)
docket = docket.reset_index(drop=True)
print(f"There are {docket.judge.notnull().sum()} event with judge's name specified in docket text")

There are 119674 event with judge's name specified in docket text


In [17]:
judge_in_first_event = docket.drop_duplicates(subset=["Case_ID"], keep="first")["judge"]
index_of_missing_judge_first_event = judge_in_first_event[judge_in_first_event.isnull()]

In [18]:
docket.loc[index_of_missing_judge_first_event.index, "judge"] = "MISSING"

In [19]:
docket["judge"].fillna(method='ffill', inplace=True)

In [20]:
docket.drop(columns="Docket_Text", inplace=True)
docket.reset_index(inplace=True, drop=True)
docket.head()

Date          Case_ID  fta  \
0 2018-01-01  2018 CMD 000009    0   
1 2018-01-01  2018 CMD 000009    0   
2 2018-01-01  2018 CMD 000009    0   
3 2018-01-01  2018 CMD 000009    0   
4 2018-01-01  2018 CMD 000009    0   

                                         charge_type           judge  
0  Sexual Solicitation Charge 1: Sexual Solicitation         MISSING  
1                                Sexual Solicitation         MISSING  
2                                                NaN         MISSING  
3                                                NaN  COURTROOM C-10  
4                                                NaN  COURTROOM C-10

In [21]:
# Now we use featuretools to create entities. 
# Note that we are including fta as a feature.
# That should be fine, because we also have "Date" set
# as a TimeIndex, which will only allow us to use 
# PRIOR ftas -- that is ftas that occur prior to start of 
# our current case to predict an FTA in the current case. 

# Variable type will change which functions are automatically applied to generate features.

es = ft.EntitySet(id="Cases")

pvts = {
    'Date': ft.variable_types.TimeIndex,
    'Case_ID': ft.variable_types.Categorical,
    'fta': ft.variable_types.Boolean,
    "charge_type": ft.variable_types.Categorical,
    "judge": ft.variable_types.Categorical
}

es = es.entity_from_dataframe(
    entity_id = "docket", 
    dataframe = docket,
    variable_types = pvts, 
    index = 'docket_index',
)

es

2019-11-21 02:43:18,413 featuretools.entityset - WARNING    index docket_index not found in dataframe, creating new integer column


Entityset: Cases
  Entities:
    docket [Rows: 635859, Columns: 6]
  Relationships:
    No relationships

In [22]:
labels = docket[docket.fta==1][['Case_ID','fta']]
cases = summaries.copy()
cases.index = cases.Case_ID.copy()
cases = cases.join(labels.set_index('Case_ID')).copy()
summaries = cases.drop_duplicates(subset='Case_ID',keep='first')

vts = {'Case_ID': ft.variable_types.Index, 
       'Case_Type': ft.variable_types.Categorical,
       'File_Date': ft.variable_types.TimeIndex, 
}

es = es.entity_from_dataframe(
    entity_id = "summaries", 
    dataframe = summaries,
    index = 'Case_ID',
    time_index="File_Date",
    variable_types=vts)

es = es.add_relationship(
    ft.Relationship(
        es["summaries"]["Case_ID"],
        es["docket"]["Case_ID"]
    )
)

es

Entityset: Cases
  Entities:
    docket [Rows: 635859, Columns: 6]
    summaries [Rows: 17159, Columns: 5]
  Relationships:
    docket.Case_ID -> summaries.Case_ID

In [23]:
# now lets pull labels from the docket

labels = docket.iloc[np.where(docket["fta"].values == 1)].drop_duplicates(subset="Case_ID", keep="first")[["Case_ID", "fta"]].reset_index(drop=True)
labels

Case_ID  fta
0     2018 CMD 000009    1
1     2018 CMD 000019    1
2     2018 CMD 000078    1
3     2018 CMD 000092    1
4     2018 CMD 000112    1
...               ...  ...
5286  2016 CMD 021145    1
5287  2016 CMD 021147    1
5288  2016 CMD 021228    1
5289  2016 CMD 021249    1
5290  2016 CMD 021306    1

[5291 rows x 2 columns]

In [24]:
pvts = {
    'fta': ft.variable_types.Text,
    'Case_ID': ft.variable_types.Categorical,
}

es = es.entity_from_dataframe(
    entity_id = "labels", 
    dataframe = labels,
    variable_types = pvts, 
    index = 'Case_ID',
)

es = es.add_relationship(
    ft.Relationship(
        es["summaries"]["Case_ID"],
        es["labels"]["Case_ID"]
    )
)

es

Entityset: Cases
  Entities:
    docket [Rows: 635859, Columns: 6]
    summaries [Rows: 17159, Columns: 5]
    labels [Rows: 5291, Columns: 2]
  Relationships:
    docket.Case_ID -> summaries.Case_ID
    labels.Case_ID -> summaries.Case_ID

In [25]:
receipts_files = DATA_PATH.glob("CMD_receipts*.csv")
receipts = pd.concat((pd.read_csv(f, index_col='Receipt_Number') for f in receipts_files), ignore_index=True)
receipts = receipts[receipts["Case_ID"].isin(subcase)]
receipts = receipts.drop(columns=['Receipts_Index', 'Received_From']).copy()
receipts['Payment_Amount'] = pd.to_numeric(receipts['Payment_Amount'].str.replace('$', ''), errors='coerce')
receipts['Receipt_Date'] = pd.to_datetime(receipts['Receipt_Date'], errors='coerce')

vts = {'Receipt_Number': ft.variable_types.Index, 
       'Receipt_Date': ft.variable_types.TimeIndex, 
       'Case_ID': ft.variable_types.Categorical, 
       'Payment_Amount': ft.variable_types.Numeric, 
}
es = es.entity_from_dataframe(entity_id = "receipts", 
                             dataframe = receipts,
                             index="Receipt_Number",
                             time_index="Receipt_Date",
                             variable_types=vts,
)

es = es.add_relationship(
    ft.Relationship(
        es["summaries"]["Case_ID"],
        es["receipts"]["Case_ID"]
    )
)

es

2019-11-21 02:43:18,986 featuretools.entityset - WARNING    index Receipt_Number not found in dataframe, creating new integer column


Entityset: Cases
  Entities:
    docket [Rows: 635859, Columns: 6]
    summaries [Rows: 17159, Columns: 5]
    labels [Rows: 5291, Columns: 2]
    receipts [Rows: 3880, Columns: 4]
  Relationships:
    docket.Case_ID -> summaries.Case_ID
    labels.Case_ID -> summaries.Case_ID
    receipts.Case_ID -> summaries.Case_ID

In [26]:
# Already have party dataframe before
parties = party_df.drop(columns=['party_type', 'party_akas']).copy()

pvts = {
    'party_label': ft.variable_types.FullName,
    'party_attorney': ft.variable_types.FullName,
    'Case_ID': ft.variable_types.Categorical,
}

es = es.entity_from_dataframe(
    entity_id = "parties", 
    dataframe = parties,
    variable_types = pvts, 
    make_index = True, 
    index = 'parties_index',
)

es.normalize_entity(base_entity_id='parties', new_entity_id='consumers', index='party_label')
es.normalize_entity(base_entity_id='parties', new_entity_id='attorneys', index='party_attorney')

es = es.add_relationship(
    ft.Relationship(
        es["summaries"]["Case_ID"],
        es["parties"]["Case_ID"]
    )
)



In [27]:
es.plot()

# generate cutoff date for each label 
We can't use any data after the start of a case to predict the likelihood of FTA in the case, 
so we use the case file date as the "cutoff" for all data allowed in any cases prediction. 

In [28]:
# Take the index and the case file date to use as a cutoff time
cutoff_times = es['summaries'].df[['Case_ID', 'File_Date', 'fta']].sort_values(by='File_Date')

# Rename columns to avoid confusion
cutoff_times.rename(columns = {'File_Date': 'cutoff_time', 
                               'fta': 'label'},
                    inplace = True)
cutoff_times.shape

(17159, 3)

In [29]:
%%time
# Generate features using the constructed entityset
fm, features = ft.dfs(
    entityset=es,
    target_entity='summaries',
    agg_primitives=[
        "count",
        "sum",
        'percent_true',
        "avg_time_between",
        "time_since_last",
        "time_since_first",
        "num_unique",
    ],
    trans_primitives=[
        "cum_count", 
        "percentile", 
        "year", 
        "month",
    ],
    max_depth = 1,
    approximate='12h',
#     cutoff_time = cutoff_times.sample(1000), # just test on 10000 cases
    verbose = False,
#     n_jobs = cpus-4,
)


CPU times: user 23.5 s, sys: 320 ms, total: 23.8 s
Wall time: 23.9 s


# Encode categoricals as booleans
FeatureTools doesn't do this automatically bc not every ML algo requires it.  
So we do it here. We just fead in the feature matrix matrix (fm) and features.  
One hot encoding creates very large sparse data for things like charges. 
So we should reduce the possible number to 100 or things will get really big. 


In [30]:
fm_enc, fs_enc = ft.encode_features(
    fm, features, top_n=100
)

In [31]:
fs_enc

[<Feature: Hold_Out>,
 <Feature: fta>,
 <Feature: Case_Type = Misdemeanor>,
 <Feature: Case_Type is unknown>,
 <Feature: COUNT(docket)>,
 <Feature: PERCENT_TRUE(docket.fta)>,
 <Feature: NUM_UNIQUE(docket.charge_type)>,
 <Feature: NUM_UNIQUE(docket.judge)>,
 <Feature: COUNT(receipts)>,
 <Feature: SUM(receipts.Payment_Amount)>,
 <Feature: AVG_TIME_BETWEEN(receipts.Receipt_Date)>,
 <Feature: TIME_SINCE_LAST(receipts.Receipt_Date)>,
 <Feature: TIME_SINCE_FIRST(receipts.Receipt_Date)>,
 <Feature: COUNT(parties)>,
 <Feature: NUM_UNIQUE(parties.party_attorney)>,
 <Feature: NUM_UNIQUE(parties.party_label)>,
 <Feature: PERCENTILE(fta)>,
 <Feature: YEAR(File_Date) = 2012>,
 <Feature: YEAR(File_Date) = 2013>,
 <Feature: YEAR(File_Date) = 2014>,
 <Feature: YEAR(File_Date) = 2016>,
 <Feature: YEAR(File_Date) = 2010>,
 <Feature: YEAR(File_Date) = 2017>,
 <Feature: YEAR(File_Date) = 2011>,
 <Feature: YEAR(File_Date) = 2018>,
 <Feature: YEAR(File_Date) = 2009>,
 <Feature: YEAR(File_Date) = 2015>,
 <Fe

In [32]:
fm_enc.head()

Hold_Out  fta  Case_Type = Misdemeanor  Case_Type is unknown  \
Case_ID                                                                         
2007 CMD 000561     False  NaN                        1                     0   
2007 CMD 001493     False  NaN                        1                     0   
2007 CMD 001496     False  NaN                        1                     0   
2007 CMD 000047     False  NaN                        1                     0   
2007 CMD 000061     False  1.0                        1                     0   

                 COUNT(docket)  PERCENT_TRUE(docket.fta)  \
Case_ID                                                    
2007 CMD 000561             29                  0.000000   
2007 CMD 001493             13                  0.000000   
2007 CMD 001496             10                  0.000000   
2007 CMD 000047             26                  0.000000   
2007 CMD 000061             54                  0.018519   

                 NUM_UNIQUE(docket.charge_type)  NUM_UNIQUE(docket.judge)  \
Case_ID                                                                     
2007 CMD 000561                               1                         3   
2007 CMD 001493                               1                         2   
2007 CMD 001496                               1                         2   
2007 CMD 000047                               1                         2   
2007 CMD 000061                               1                         2   

                 COUNT(receipts)  SUM(receipts.Payment_Amount)  ...  \
Case_ID                                                         ...   
2007 CMD 000561              0.0                           0.0  ...   
2007 CMD 001493              0.0                           0.0  ...   
2007 CMD 001496              0.0                           0.0  ...   
2007 CMD 000047              0.0                           0.0  ...   
2007 CMD 000061              0.0                           0.0  ...   

                 MONTH(File_Date) = 4  MONTH(File_Date) = 8  \
Case_ID                                                       
2007 CMD 000561                     0                     0   
2007 CMD 001493                     0                     0   
2007 CMD 001496                     0                     0   
2007 CMD 000047                     0                     0   
2007 CMD 000061                     0                     0   

                 MONTH(File_Date) = 7  MONTH(File_Date) = 1  \
Case_ID                                                       
2007 CMD 000561                     0                     0   
2007 CMD 001493                     0                     0   
2007 CMD 001496                     0                     0   
2007 CMD 000047                     0                     1   
2007 CMD 000061                     0                     1   

                 MONTH(File_Date) = 6  MONTH(File_Date) = 9  \
Case_ID                                                       
2007 CMD 000561                     0                     0   
2007 CMD 001493                     0                     0   
2007 CMD 001496                     0                     0   
2007 CMD 000047                     0                     0   
2007 CMD 000061                     0                     0   

                 MONTH(File_Date) = 2  MONTH(File_Date) = 11  \
Case_ID                                                        
2007 CMD 000561                     0                      1   
2007 CMD 001493                     0                      0   
2007 CMD 001496                     0                      0   
2007 CMD 000047                     0                      0   
2007 CMD 000061                     0                      0   

                 MONTH(File_Date) = 12  MONTH(File_Date) is unknown  
Case_ID                                                              
2007 CMD 000561                      0                            0  
2007 

# Create test-train split. 
It's possible that this should be handled in the ML notebook, but I prefer to handle it here. 

In [33]:
holdout_data = fm_enc[fm_enc['Hold_Out']==True]

In [34]:
holdout_data['label'] = holdout_data['fta'].fillna(value=0).copy()
tt_data = fm_enc[fm_enc['Hold_Out']==False]

In [35]:
print(tt_data.columns)


tt_data['label'] = tt_data['fta'].fillna(value=0).copy()

Index(['Hold_Out', 'fta', 'Case_Type = Misdemeanor', 'Case_Type is unknown',
       'COUNT(docket)', 'PERCENT_TRUE(docket.fta)',
       'NUM_UNIQUE(docket.charge_type)', 'NUM_UNIQUE(docket.judge)',
       'COUNT(receipts)', 'SUM(receipts.Payment_Amount)',
       'AVG_TIME_BETWEEN(receipts.Receipt_Date)',
       'TIME_SINCE_LAST(receipts.Receipt_Date)',
       'TIME_SINCE_FIRST(receipts.Receipt_Date)', 'COUNT(parties)',
       'NUM_UNIQUE(parties.party_attorney)', 'NUM_UNIQUE(parties.party_label)',
       'PERCENTILE(fta)', 'YEAR(File_Date) = 2012', 'YEAR(File_Date) = 2013',
       'YEAR(File_Date) = 2014', 'YEAR(File_Date) = 2016',
       'YEAR(File_Date) = 2010', 'YEAR(File_Date) = 2017',
       'YEAR(File_Date) = 2011', 'YEAR(File_Date) = 2018',
       'YEAR(File_Date) = 2009', 'YEAR(File_Date) = 2015',
       'YEAR(File_Date) = 2019', 'YEAR(File_Date) = 2007',
       'YEAR(File_Date) = 2008', 'YEAR(File_Date) = 2006',
       'YEAR(File_Date) is unknown', 'MONTH(File_Date) = 3',
    

In [36]:
len(holdout_data)

1099

In [37]:
tt_data.reset_index(inplace=True)

tt_data.drop(["PERCENT_TRUE(docket.fta)", "Hold_Out", "Case_ID", "fta", "PERCENTILE(fta)"], axis=1, inplace=True)
X = tt_data.reset_index(drop=True)
X = X.fillna(0)
y = X.pop("label")
X_train, X_test, y_train, y_test = train_test_split(
    X.values, y.values, train_size=0.75, test_size=0.25
)

In [38]:
X.columns

Index(['Case_Type = Misdemeanor', 'Case_Type is unknown', 'COUNT(docket)',
       'NUM_UNIQUE(docket.charge_type)', 'NUM_UNIQUE(docket.judge)',
       'COUNT(receipts)', 'SUM(receipts.Payment_Amount)',
       'AVG_TIME_BETWEEN(receipts.Receipt_Date)',
       'TIME_SINCE_LAST(receipts.Receipt_Date)',
       'TIME_SINCE_FIRST(receipts.Receipt_Date)', 'COUNT(parties)',
       'NUM_UNIQUE(parties.party_attorney)', 'NUM_UNIQUE(parties.party_label)',
       'YEAR(File_Date) = 2012', 'YEAR(File_Date) = 2013',
       'YEAR(File_Date) = 2014', 'YEAR(File_Date) = 2016',
       'YEAR(File_Date) = 2010', 'YEAR(File_Date) = 2017',
       'YEAR(File_Date) = 2011', 'YEAR(File_Date) = 2018',
       'YEAR(File_Date) = 2009', 'YEAR(File_Date) = 2015',
       'YEAR(File_Date) = 2019', 'YEAR(File_Date) = 2007',
       'YEAR(File_Date) = 2008', 'YEAR(File_Date) = 2006',
       'YEAR(File_Date) is unknown', 'MONTH(File_Date) = 3',
       'MONTH(File_Date) = 5', 'MONTH(File_Date) = 10', 'MONTH(File_Date) = 4

# Save the data for analysis. 
We don't want to analyze here, just create our data for analysis. 

In [39]:
data = [X_train, X_test, y_train, y_test]
with open('fta_tt.dat', "wb") as f:
    pickle.dump(data, f)

holdout_data.to_csv('fta_holdout.csv')